### 카운트 벡터(Count Vector)

In [2]:
text = ["I am a elementary school student", "And I am a boy"]

from nltk import word_tokenize
text_tokenzied = [word_tokenize(sentence) for sentence in text]
print(text_tokenzied)

[['I', 'am', 'a', 'elementary', 'school', 'student'], ['And', 'I', 'am', 'a', 'boy']]


문장 먼저 토큰화하고, 단어의 빈도수를 세어 벡터로 만드는 방법입니다.

In [4]:
from collections import Counter

vocab_counter = Counter()
for sentence in text_tokenzied:
    vocab_counter.update(sentence)
    
print(vocab_counter)

Counter({'I': 2, 'am': 2, 'a': 2, 'elementary': 1, 'school': 1, 'student': 1, 'And': 1, 'boy': 1})


In [7]:
text_count_vector = []

for sentence in text_tokenzied:
    sentence_vector = []
    
    for word in sentence:
        sentence_vector.append(vocab_counter[word])
    text_count_vector.append(sentence_vector)
    
print(text_count_vector)

[[2, 2, 2, 1, 1, 1], [1, 2, 2, 2, 1]]


문장들을 원하는 카운터 벡터로 변환합니다. 여기서 각 단어들이 빈도수로 대체되는 것을 확인할 수 있습니다.

# TF-IDF(Term Frequency-Inverse Document Frequency)

'TF-IDF'의 기본적인 개념은 "위의 단순 빈도수 문제를 개선하기위해 단어의 빈도수에 역 빈도수를 곱해준다"는 것입니다.

 

### 1. TF(d, f) : 특정 문서 d에서의 특정 단어 t의 등장 횟수

     예를 들어 문서 [I am a great great boy]가 있을 때 각 단어의 'TF'는 [I, am, a , great, boy] == [1, 1, 1, 2, 1]이 됩니다.

 

### 2. df(t) : 특정 단어 t가 등장한 문서의 수

     예를들어 문서 1 [I am a great great boy], 문서 2 [I am a boy]가 있을 때 'df'는 [I, am, a , great, boy] == [2, 2, 2, 1, 2]이 됩니다. 여기서 great == 1로 정의되는데 그 이유는 great이 문서 1에서 2번 사용되었지만, 사용된 문서수는 문서 1 하나이므로 1로 대응됩니다.

 

### 3. idf(d, t) : df(t)에 반비례하는 수
    
     idf(d, t) = log(n / (1 + df(t))) 이며, n은 전체 문서의 수입니다.

In [8]:
text = ["I am a great great elementary school student", "And I am a boy"]

# Word tokenized sentence
from nltk import word_tokenize

text_tokenzied = [word_tokenize(sentence) for sentence in text]

In [9]:
# Remove stopwords that are too short.
text_tokenzied2 = []
for sentence in text_tokenzied:
    sent = []
    for word in sentence:
        if len(word) >= 2:
            sent.append(word)
    text_tokenzied2.append(sent)

print(text_tokenzied2)

[['am', 'great', 'great', 'elementary', 'school', 'student'], ['And', 'am', 'boy']]


In [10]:
# Vocabulary list
from collections import Counter

vocab_counter = Counter()
for sentence in text_tokenzied2:
    vocab_counter.update(sentence)

vocab = []
for key, value in vocab_counter.items():
    vocab.append(key)
    
print(vocab)

['am', 'great', 'elementary', 'school', 'student', 'And', 'boy']


In [11]:
# Count word of each sentence
from collections import Counter

count = []
for sentence in text_tokenzied2:
    vocab_counter = Counter()
    vocab_counter.update(sentence)
    count.append(vocab_counter)
print(count)

[Counter({'great': 2, 'am': 1, 'elementary': 1, 'school': 1, 'student': 1}), Counter({'And': 1, 'am': 1, 'boy': 1})]



 'Counter()'를 이용해 각 문서별로 단어수들을 취합합니다. 위와 같이 문서 2개에 해당하는 count라는 list가 만들어진 것을 보실 수 있습니다. 

In [12]:
def TF(vocab, counter):
    vector = []
    for word in vocab:
        if counter[word] != False:
            vector.append(counter[word])
        else:
            vector.append(0)
    return vector

print(vocab)
print(TF(vocab, count[0]))

['am', 'great', 'elementary', 'school', 'student', 'And', 'boy']
[1, 2, 1, 1, 1, 0, 0]


In [13]:
def DF(text_tokenzied2, vocab):
    text = []
    for sentence in text_tokenzied2:
        for word in list(set(sentence)):
            text.append(word)
    vocab_counter = Counter()
    vocab_counter.update(text)
    
    df = []
    for word in vocab:
        df.append(vocab_counter[word])
    return df

print(vocab)
print(DF(text_tokenzied2, vocab))

['am', 'great', 'elementary', 'school', 'student', 'And', 'boy']
[2, 1, 1, 1, 1, 1, 1]


In [21]:
import math

def IDF(df, n):
    idf = []
    for i in df:
        idf.append(math.log((n)/(i+1))+1)
    return idf

print(vocab)
print(IDF(DF(text_tokenzied2, vocab), len(text_tokenzied2)))

['am', 'great', 'elementary', 'school', 'student', 'And', 'boy']
[0.5945348918918356, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [25]:

c = count[1]

def TFIDF(tf, idf):
    product = [x*y for x, y in zip(tf, idf)]
    return product

print(vocab)
print(TFIDF(TF(vocab, c), IDF(DF(text_tokenzied2, vocab), len(text_tokenzied2))))

['am', 'great', 'elementary', 'school', 'student', 'And', 'boy']
[0.5945348918918356, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0]


In [26]:
tfidf = []
for c in count:
    tfidf.append(TFIDF(TF(vocab, c), IDF(DF(text_tokenzied2, vocab), len(text_tokenzied2))))

print(vocab)
print(tfidf)

['am', 'great', 'elementary', 'school', 'student', 'And', 'boy']
[[0.5945348918918356, 2.0, 1.0, 1.0, 1.0, 0.0, 0.0], [0.5945348918918356, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0]]


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfv = TfidfVectorizer().fit(text)
print(tfidfv.transform(text).toarray())
print(tfidfv.vocabulary_)

[[0.25969799 0.         0.         0.36499647 0.72999294 0.36499647
  0.36499647]
 [0.44943642 0.6316672  0.6316672  0.         0.         0.
  0.        ]]
{'am': 0, 'great': 4, 'elementary': 3, 'school': 5, 'student': 6, 'and': 1, 'boy': 2}
